## Part I - Write Collected Data into PostgreSQL

### Import the functions written in the query_helper.py file

In [3]:
!pip install regex

In [4]:
!pip install psycopg2

  Using cached psycopg2-2.7.3.1-cp36-cp36m-manylinux1_x86_64.whl


In [5]:
import psycopg2 as pg2
import pandas as pd
import re

In [6]:
from os import chdir
chdir('/home/jovyan/')

In [7]:
import library.db_helper as db
import library.functions as fy

In [8]:
con, cur = db.connect_to_db()

### Insert content into the created tables (which are currently empty)

#### WRITE INTO THE POSTGRES category TABLE

In [23]:
CATEGORY_df = pd.read_pickle('./data/CATEGORY_df.p')

In [24]:
CATEGORY_df.reset_index(inplace=True, drop=True)

In [25]:
CATEGORY_df.head()

,category,categoryid
0,Machine Learning,1
1,Business Software,0


In [26]:
# clean out the category db that was previously created 
delete_cat_query = '''
BEGIN;
DELETE FROM category;
COMMIT;
'''

cur.execute(delete_cat_query)

In [29]:
for i in CATEGORY_df.index:
    db.insert_category(CATEGORY_df.iloc[i,1], CATEGORY_df.iloc[i,0])   

In [30]:
query = '''
SELECT * 
FROM category
'''

test_category_query = db.query_to_dataframe(query)

In [31]:
test_category_query

,cid,name
0,1,Machine Learning
1,0,Business Software


#### WRITE INTO THE POSTGRES page TABLE

In [32]:
PAGE_df = pd.read_pickle('./data/PAGE_df.p')

In [33]:
PAGE_df.shape

(2449, 3)

In [39]:
PAGE_df.sample(5)

,pageid,title,text
1059,2370618,lotus marketplace,lotus marketplace was a database program devel...
658,48673924,dynamic unobserved effects model,this article has multiple issues please help i...
1394,3686819,x com interceptor,x com interceptordeveloper s microprosepublish...
1716,880878,fpml,fpml financial products markup language is an ...
1542,16370912,portable flight planning software,this article has multiple issues please help i...


In [34]:
PAGE_df['title'] = PAGE_df['title'].apply(lambda x: fy.text_cleaner(x))

In [35]:
delete_page_query = '''
BEGIN;
DELETE FROM page;
COMMIT;
'''

cur.execute(delete_page_query)

In [40]:
for i in PAGE_df.index:
    db.insert_page(PAGE_df.iloc[i,0], PAGE_df.iloc[i,1], PAGE_df.iloc[i,2])

In [41]:
query = '''
SELECT * 
FROM page
'''

test_page_query = db.query_to_dataframe(query)

In [42]:
test_page_query.shape

(2449, 3)

In [44]:
test_page_query.sample(5)

,pid,text,title
1855,44739552,this article relies largely or entirely on a s...,oracle health sciences
147,40946774,in theoretical computer science a pattern lang...,pattern language formal languages
1521,8391734,this article has multiple issues please help i...,unit
1215,50515193,workforce optimization wfo is a business strat...,workforce optimization
805,20768311,pierre baldibornrome italyresidenceusafieldsco...,pierre baldi


#### WRITE INTO THE POSTGRES category_page JOIN TABLE

In [54]:
TOTAL_df = pd.read_pickle('./data/total_df.p')

In [55]:
TOTAL_df.sample(5)

,pageid,title,category,text
1804,7750889,Workrave,Business Software,workraveinitial release years ago stable rel...
799,10371071,Peter Flach,Machine Learning,peter flachbornpieter adriaan flachsneekinstit...
1842,44739552,Oracle Health Sciences,Business Software,this article relies largely or entirely on a s...
1623,31039666,Test automation management tools,Business Software,this article needs additional citations for ve...
2418,31216734,Metro 2 format,Business Software,the topic of this article may not meet wikiped...


In [56]:
from sklearn.preprocessing import LabelEncoder

In [57]:
le = LabelEncoder()

In [58]:
TOTAL_df['cid'] = le.fit_transform(TOTAL_df['category'])

In [59]:
TOTAL_df.sample(5)

,pageid,title,category,text,cid
494,47518866,Fluentd,Machine Learning,fluentddeveloper s treasure datainitial releas...,1
1722,6814048,LiveWire Professional,Business Software,it is proposed that this article be deleted be...,0
2190,33104468,Trello,Business Software,this article relies too much on references to ...,0
958,32958985,DaySmart Software,Business Software,daysmart software inc typeprivate companyindus...,0
1632,2732586,Zimbra,Business Software,this article relies too much on references to ...,0


In [60]:
delete_catpage_query = '''
BEGIN;
DELETE from category_page;
COMMIT;
'''

cur.execute(delete_catpage_query)

In [61]:
CATEGORY_PAGE_df = TOTAL_df[['pageid', 'cid']]

In [62]:
CATEGORY_PAGE_df.iloc[0,0], CATEGORY_PAGE_df.iloc[0,1]

(54972729, 1)

In [63]:
for i in CATEGORY_PAGE_df.index:
    db.insert_category_page(CATEGORY_PAGE_df.iloc[i,0], CATEGORY_PAGE_df.iloc[i,1])

In [64]:
query = '''
SELECT *
FROM category_page
'''

catpage_test_query = db.query_to_dataframe(query)

In [65]:
catpage_test_query.shape

(2449, 2)

In [66]:
catpage_test_query

,category_cid,page_pid
0,1,54972729
1,1,43385931
2,1,49082762
3,1,233488
4,1,53587467
5,1,3771060
6,1,43808044
7,1,28801798
8,1,45049676
9,1,52642349
